In [729]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn.metrics import accuracy_score, classification_report


In [730]:
import pandas as pd # data processing
import seaborn as sb # visualizations
import matplotlib.pyplot as plt # editing visualizations
from matplotlib import style # setting styles for plots
from sklearn.preprocessing import StandardScaler # normalizing data
from sklearn.neighbors import KNeighborsClassifier # KNN algorithm
from sklearn.metrics import accuracy_score # algorithm accuracy
from sklearn.model_selection import train_test_split # splitting the data

In [731]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import neighbors



In [732]:
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler


In [733]:
df_06 = pd.read_csv('datasets/htx_2006_weather.csv', na_values='Blank').dropna()
df_10 = pd.read_csv('datasets/htx_2010_weather.csv', na_values='Blank').dropna()
df_11 = pd.read_csv('datasets/htx_2011_weather.csv', na_values='Blank').dropna()
df_12 = pd.read_csv('datasets/htx_2012_weather.csv', na_values='Blank').dropna() 
df_13 = pd.read_csv('datasets/htx_2013_weather.csv', na_values='Blank').dropna()
df_14 = pd.read_csv('datasets/htx_2014_weather.csv', na_values='Blank').dropna()
df_15 = pd.read_csv('datasets/htx_2015_weather.csv', na_values='Blank').dropna()
df_18 = pd.read_csv('datasets/htx_2018_weather.csv', na_values='Blank').dropna()
df_19 = pd.read_csv('datasets/htx_2019_weather.csv', na_values='Blank').dropna()
df_21 = pd.read_csv('datasets/htx_2021_weather.csv', na_values='Blank').dropna()

df = pd.concat([df_06, df_10, df_11, df_12, df_13, df_14, df_15, df_18, df_19, df_21])
# df = df_06

In [734]:
# remap Yes/No to 1/0 for rain_tomorrow column
df.rain_tomorrow = df.rain_tomorrow.map(dict(Yes=1, No=0))

In [735]:
df.rain_tomorrow.value_counts()

0    2955
1     662
Name: rain_tomorrow, dtype: int64

In [736]:
rainfall = df['rainfall'].to_numpy()
rain_tomorrow = df['rain_tomorrow'].to_numpy()


In [737]:
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.6*len(df)), int(.8*len(df))])

In [738]:
test

,date,min_temp,max_temp,rainfall,wind_speed9am,wind_speed3pm,humidity9am,humidity3pm,pressure9am,pressure3pm,cloud9am,cloud3pm,temp9am,temp3pm,rain_today,rain_tomorrow
319,2021-11-16,64.0,83.0,0.0,10.0,12.0,71.0,47.0,30.06,29.97,Fair,Partly Cloudy,74.0,83.0,No,0
232,2015-08-21,74.0,88.0,1.6,5.0,0.0,93.0,55.0,29.99,29.88,Light Rain,Mostly Cloudy,74.0,88.0,Yes,0
194,2014-07-14,78.0,92.0,0.0,9.0,15.0,67.0,59.0,29.99,29.92,Partly Cloudy,Mostly Cloudy,85.0,90.0,No,0
271,2015-09-29,70.0,88.0,0.1,10.0,10.0,76.0,53.0,29.80,29.74,Partly Cloudy,Mostly Cloudy,77.0,88.0,Yes,0
156,2014-06-06,75.0,89.0,0.0,9.0,10.0,69.0,61.0,29.90,29.89,Partly Cloudy,Partly Cloudy,84.0,87.0,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,2012-02-25,46.0,56.0,0.0,5.0,7.0,40.0,37.0,30.36,30.29,Cloudy,Cloudy,52.0,55.0,No,0
219,2012-08-07,78.0,95.0,0.0,5.0,7.0,72.0,47.0,29.94,29.87,Partly Cloudy,Mostly Cloudy,85.0,95.0,No,1
149,2011-05-30,78.0,91.0,0.0,18.0,17.0,59.0,50.0,29.92,29.90,Partly Cloudy,Partly Cloudy,87.0,90.0,No,0
254,2021-09-12,75.0,91.0,0.0,9.0,12.0,61.0,43.0,30.03,29.98,Fair,Mostly Cloudy,85.0,89.0,No,1


In [739]:
x_train = train[[
                'min_temp', 
                'max_temp', 
                'wind_speed3pm', 
                'humidity3pm', 
                'humidity9am',
                'pressure3pm', 
                'pressure9am',
                'temp9am',
                'temp3pm',
                'rainfall'
                ]]

y_train = train[['rain_tomorrow']]

In [740]:
from imblearn.over_sampling import RandomOverSampler
over_sampler = RandomOverSampler(sampling_strategy='minority', random_state=42)
x_train_resampled, y_train_resampled = over_sampler.fit_resample(x_train, y_train)

In [741]:
# rebalance the data to balance the rain_tomorrow 1s and 0s
x_train_resampled, y_train_resampled = over_sampler.fit_resample(x_train, y_train)
x_train = x_train_resampled
y_train = y_train_resampled

y_train.value_counts()
x_val = validate[x_train.columns].copy()
y_val = validate[y_train.columns].copy()



In [742]:
x_test = test[[
                'min_temp', 
                'max_temp', 
                'wind_speed3pm',
                'humidity3pm', 
                'humidity9am',
                'pressure3pm',
                'pressure9am', 
                'temp9am',
                'temp3pm',
                'rainfall'
                ]]

y_test = test[['rain_tomorrow']]

In [743]:
model = KNeighborsClassifier() 
model.fit(x_train, y_train)

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [744]:
pred = model.predict(x_test)

In [745]:
# x_train, x_test, y_train, y_test = train_test_split( test_size=0.33, random_state=41) 
n_neighbors = 10
h           = .02
X = df[['min_temp', 
        'max_temp', 
        'wind_speed3pm', 
        'humidity3pm', 
        'humidity9am',
        'pressure3pm', 
        'pressure9am',
        'temp9am',
        'temp3pm',
        'rainfall']]
y = df[['rain_tomorrow']]
cmap_light = ListedColormap(['#FFAAAA', '#AAAAFF'])
cmap_bold  = ListedColormap(['#FF0000', '#0000FF'])

print(X)
print(y)

for weights in ['uniform', 'distance']:
  clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
  clf.fit(x_train, y_train)

  x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
  y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
  xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
  Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

     min_temp  max_temp  wind_speed3pm  humidity3pm  humidity9am  pressure3pm  \
0        64.0      81.0           14.0         54.0         90.0        29.65   
1        57.0      79.0           10.0         22.0         87.0        29.89   
2        49.0      79.0           10.0         24.0         60.0        29.98   
3        52.0      76.0            7.0         60.0         87.0        29.97   
4        51.0      69.0           23.0         21.0         42.0        30.13   
..        ...       ...            ...          ...          ...          ...   
360      68.0      81.0           16.0         58.0         87.0        29.79   
361      72.0      84.0           15.0         53.0         74.0        29.63   
362      72.0      84.0           12.0         56.0         85.0        29.67   
363      67.0      83.0            9.0         58.0         84.0        29.79   
364      73.0      85.0           17.0         58.0         87.0        29.71   

     pressure9am  temp9am  

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:233: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


TypeError: '(slice(None, None, None), 0)' is an invalid key

In [ ]:
print('R squared: {:.2f}'.format(model.score(x_train,y_train)*100))

R squared: 85.94


In [ ]:
print('R squared: {:.2f}'.format(model.score(x_test,y_test)*100))

R squared: 65.33


In [ ]:
print('R squared: {:.2f}'.format(model.score(x_val,y_val)*100))

R squared: 60.58
